In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import resgen as rg

rgc = rg.connect(os.getenv('RESGEN_USER'), os.getenv('RESGEN_PASSWORD'))

In [6]:
project = rgc.find_or_create_project('Mumbach et al. 2016', group='paper-data')

In [35]:
project.sync_dataset('GSE80820_HiChIP_mES_Oct4.mcool', assembly='mm9')
project.sync_dataset('GSE80820_HiChIP_GM_cohesin.mcool', assembly='hg19')
project.sync_dataset('GSE80820_HiChIP_mES_cohesin_all.mcool', assembly='mm9')

Inferred filetype: cooler
Inferred datatype: matrix
Inferred filetype: cooler
Inferred datatype: matrix
Inferred filetype: cooler
Inferred datatype: matrix


BfHPm0qW: GSE80820_HiChIP_mES_cohesin_all.mcool

In [38]:
ds = project.sync_dataset(
    'HiCCUPS_mES_cohesin_all/GSE80820_HiChIP_hiccups_loops_mES_cohesin_all.beddb',
    assembly='mm9',
    datatype='bedlike',
    filetype='beddb'
)

In [47]:
%%javascript

require(["https://unpkg.com/higlass-arcs@0.1.7/dist/higlass-arcs.min.js"],
    function(hglib) {
});

<IPython.core.display.Javascript object>

In [40]:
tr = ds.hg_track(track_type='1d-arcs',
                 position='top', height=100)

In [41]:
datasets = rgc.find_datasets(project=project, limit=20)
ds_dict = dict([(d.name, d) for d in datasets])
ds_dict

{'GSE80820_HiChIP_mES_Oct4.mcool': ZNC7GBb7: GSE80820_HiChIP_mES_Oct4.mcool,
 'GSE80820_HiChIP_GM_cohesin.mcool': KZaf4npb: GSE80820_HiChIP_GM_cohesin.mcool,
 'GSE80820_HiChIP_mES_cohesin_all.mcool': BfHPm0qW: GSE80820_HiChIP_mES_cohesin_all.mcool,
 'GSE80820_HiChIP_hiccups_loops_mES_cohesin_all.beddb': D3QBgh1R: GSE80820_HiChIP_hiccups_loops_mES_cohesin_all.beddb}

In [42]:
chromsizes_mm9 = rgc.find_datasets(
    datatype='chromsizes', assembly='mm9'
)[0]
gene_annotations = rgc.find_datasets(
    datatype='gene-annotations', assembly='mm9'
)[0]

In [46]:
import higlass
from higlass.client import View

view = View(
    [
        chromsizes_mm9.hg_track(),
        gene_annotations.hg_track(),
        tr,
        ds_dict['GSE80820_HiChIP_mES_cohesin_all.mcool'].hg_track(
            track_type='horizontal-heatmap',
            height=200,
            labelPosition='topLeft'
        )
    ],
    initialXDomain=[0,1e8])
(d,s,v) = higlass.display([view])
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'b9KiXmsHTbOf3ahsL6o2HQ', 'tracks': {'top': [{'ty…

In [52]:
project.sync_viewconf(d.viewconf, 'HiChIP loops with heatmap and arcs')

In [57]:
rao_proj = rgc.find_project('Rao et al. 2014', group='paper-data')

In [69]:
rao_dict = dict([d.name, d] for d in rao_proj.list_datasets())
rao_dict

{'Rao2014-GM12878-MboI-allreps-filtered.mcool': J73wN2WC: Rao2014-GM12878-MboI-allreps-filtered.mcool}

In [70]:
hichip = ds_dict['GSE80820_HiChIP_GM_cohesin.mcool']
hic = rao_dict['Rao2014-GM12878-MboI-allreps-filtered.mcool']

In [74]:
chromsizes_hg19 = rgc.find_datasets(
    datatype='chromsizes', assembly='hg19'
)[0]
chrominfo_hg19 = rgc.get_chrominfo(chromsizes_hg19)

In [124]:
from higlass.utils import hg_cmap

padding = 0.05
wide_view = chrominfo_hg19.to_abs_range('chr8', 0, 147e6, padding)
middle_view = chrominfo_hg19.to_abs_range('chr8', 128e6, 142e6, padding)
narrow_view = chrominfo_hg19.to_abs_range('chr8', 133.9e6, 134.5e6, padding)

track_options = {
    'colorRange': hg_cmap('Reds'),
    'height': 200,
    'colorbarPosition': 'hidden',
    'labelPosition': 'hidden',
    'showMousePosition': 'true',
}

vp_options = {
    'projectionFillColor': 'black',
    'projectionStrokeColor': 'white',
}

from higlass.client import ViewportProjection

v_hch3= View([hichip.hg_track(**track_options)],
         width=4, x=8, initialXDomain=narrow_view)
vp3 = ViewportProjection(v_hch3, options=vp_options)

v_hch2= View([hichip.hg_track(**track_options) + vp3],
         width=4, x=4, initialXDomain=middle_view)
vp2 = ViewportProjection(v_hch2, options=vp_options)

v_hch1= View([hichip.hg_track(
            **{
                **track_options,
                "name": "HiChIP",
                "labelPosition": "topLeft"
            }) + vp2],
         width=4, initialXDomain=wide_view)

v_hic3= View([hic.hg_track(**track_options)],
         width=4, x=8, initialXDomain=narrow_view)
v_hic2= View([hic.hg_track(**track_options) + vp3],
         width=4, x=4, initialXDomain=middle_view)
v_hic1= View([hic.hg_track(
            **{
                **track_options,
                "name": "Hi-C",
                "labelPosition": "topLeft"
            }) + vp2],
         width=4, initialXDomain=wide_view)

syncs = [[v_hch1, v_hic1], [v_hch2, v_hic2], [v_hch3, v_hic3]]

(d,s,v) = higlass.display([
    v_hch1, v_hch2, v_hch3,
    v_hic1, v_hic2, v_hic3
],
    location_syncs=syncs,
    zoom_syncs=syncs,
    auth_token=f"JWT {rgc.token}")
d

HiGlassDisplay(auth_token='JWT eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJ1c2VyX2lkIjoyLCJ1c2VybmFtZSI6InBldGUiLC…

In [125]:
project.sync_viewconf(v, "Fig. 2A HiChIP vs. Hi-C")

In [136]:
extent = chrominfo_hg19.to_abs_range('chr9', 36.95e6, 37.45e6, 0.05)

track_options = {
}

hic_track =hic.hg_track(
            **{
                **track_options,
                "name": "Hi-C",
                "labelPosition": "topLeft"
            }) 

hch_track = hichip.hg_track(
            **{
                **track_options,
                "name": "HiChIP",
                "labelPosition": "topLeft"
            })
v_hic1= View([hic_track],
         width=4, x=4, initialXDomain=extent)

v_hch1= View([hch_track],
         width=4, initialXDomain=extent)
v_div = View([hch_track / hic_track],
             width=4, x=8, initialXDomain=extent)
(d,s,v) = higlass.display([v_hic1, v_hch1, v_div])
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'W6DIx9uhR42I8Wg5o7uGOA', 'tracks': {'top': [], '…